In [4]:
from bs4 import BeautifulSoup
import requests
import time
import csv
from tqdm import tqdm
from textblob import TextBlob
from transformers import pipeline
import spacy
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import re

In [10]:
# Définir les en-têtes HTTP pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

# Fonction pour scraper une page de recherche et récupérer les articles
def scrape_page(url, session):
    all_articles = []  # Liste de tous les articles
    seen_articles = set()  # Ensemble pour garder trace des articles déjà ajoutés

    try:
        # Envoyer une requête HTTP pour obtenir la page
        response = session.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Trouver tous les articles pour les deux cas
            articles = soup.find_all("article", class_=["fig-profil fig-profil-mtpd", "fig-profil"])

            for article in articles:
                # Extraire le titre
                title_element = article.find("h2", class_="fig-profil-headline")
                title_link = title_element.find("a") if title_element else None
                title_text = title_link.get_text(strip=True) if title_link else "Titre non disponible"

                # Extraire l'URL de l'article
                article_url = title_link["href"] if title_link else None

                # Extraire la description
                description_element = article.find("div", class_="fig-profil-chapo")
                description_text = description_element.get_text(strip=True) if description_element else "Description non disponible"

                # Extraire la date de publication
                date_element = article.find("li", class_="fig-date-pub")
                time_element = date_element.find("time") if date_element else None
                date_text = time_element["datetime"] if time_element else "Date non disponible"

                # Ajouter l'article à la liste si non déjà vu
                if article_url and article_url not in seen_articles:
                    all_articles.append({
                        "Titre": title_text,
                        "Description": description_text,
                        "URL": article_url,
                        "Date": date_text
                    })
                    seen_articles.add(article_url)

            # Gérer le bouton "Voir plus de résultats"
            more_button = soup.find("button", class_="fh-kw__lk fh-kw__lk--more")
            if more_button:
                # Simuler un clic pour charger plus de résultats
                next_url = more_button.get("data-url")
                if next_url:
                    print(f"Bouton 'Voir plus de résultats' détecté. Scraping de {next_url}")
                    all_articles.extend(scrape_page(next_url, session))
            
            # Gérer le panneau "Mon compte" pour charger dynamiquement les articles
            login_button = soup.find("button", class_="fh-user__login fh-user__login--is-connected")
            if login_button:
                print("Bouton 'Mon compte' détecté. Articles supplémentaires peuvent être chargés.")
                # Simuler une interaction ou vérifier si plus de contenu s'affiche dynamiquement

            return all_articles

        else:
            print(f"Erreur HTTP {response.status_code} lors de l'accès à {url}")
            return []

    except requests.RequestException as e:
        print(f"Erreur de requête pour {url}: {e}")
        return []

# URL de base de la recherche
base_url = "https://recherche.lefigaro.fr/recherche/agriculture%20bio/"

# Initialiser une session HTTP pour gérer les requêtes
with requests.Session() as session:
    # Scraper la page unique
    articles = scrape_page(base_url, session)



Bouton 'Mon compte' détecté. Articles supplémentaires peuvent être chargés.


In [11]:
len(articles)

20